In [0]:
try:
  dbutils.fs.unmount("/mnt/input")
except:
  print("Input Mount does not exist")

In [0]:
try:
  dbutils.fs.unmount("/mnt/output1")
except:
  print("Output Mount does not exist")

In [0]:
input_source = dbutils.widgets.get("input")
output1 = dbutils.widgets.get("testdata")

In [0]:
input_source

In [0]:
output1

In [0]:
#container = 'diabetes-container'
#storage_account_name = 'linkingstorage'

conf_key = "fs.azure.account.key.linkingstorage.blob.core.windows.net"
key_value = "IqiE5zUREOUTXhlAL1tpPhd3KyBjuXSyoWTT9F9TXNKpn86HSqbgaLQP/5YZRliN1SXey5lVHMswM8L347JDDw=="

#spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", key_value)

In [0]:
try: 
  dbutils.fs.mount(source = input_source,
                   mount_point = "/mnt/input",
                   extra_configs = {conf_key:key_value})
except:
  print('Input datastore already mounted...')

In [0]:
try:
  dbutils.fs.mount(source = output1,
                   mount_point = "/mnt/output1",
                   extra_configs = {conf_key:key_value})
except:
  print("Output data folder already mounted...")

In [0]:
import pandas as pd
df = pd.read_csv("/dbfs/mnt/input/diabetes.csv")

In [0]:
# Create Dummy variables
data_prep = df.iloc[:,1:]

# Create X and Y Variables
X = data_prep.iloc[:, :-1]
Y = data_prep.iloc[:, -1]

# Split the X and Y dataset into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1234, stratify=Y)

In [0]:
# Import and train Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1234)

trained_model = rfc.fit(X_train, Y_train)

# Test the RFC model
Y_predict = rfc.predict(X_test)

In [0]:
dbutils.fs.mkdirs("/mnt/output1/000")

In [0]:
#writing model file to blob storage
import joblib
obj_file = "/dbfs/mnt/output1/rfcModel.pkl"
joblib.dump(value=trained_model, filename=obj_file)

In [0]:
#Registering the same model in azure ml
#joblib.dump(value=rfc, filename="rfcModel.pkl")
#https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py
#model = Model.register(workspace=ws, model_path="rfcModel.pkl", model_name="RandomForestModel")

In [0]:
#Writing it to blob storage

X_test.to_csv("/dbfs/mnt/output1/x_test.csv", index=False)

Y_test = pd.DataFrame(Y_test)
Y_test.to_csv("/dbfs/mnt/output1/y_test.csv", index=False)
#preedicted data
Y_predict = pd.DataFrame(Y_predict)
Y_predict.to_csv("/dbfs/mnt/output1/y_predict.csv", index=False)